In [1]:
from matplotlib.animation import FuncAnimation
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm as tqdm
import pandas as pd

In [2]:
%matplotlib qt

In [3]:
save_direc = "/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/animation_data"

In [4]:
import filehandling
from particletracking import dataframes

In [5]:
flat_files = filehandling.get_directory_filenames("/media/data/Data/FirstOrder/Susceptibility/Flat2/*.hdf5")

In [6]:
dimpled_files = filehandling.get_directory_filenames("/media/data/Data/FirstOrder/Susceptibility/Dimpled2/*.hdf5")

In [7]:
def get_data_attributes(file, param):
    data = dataframes.DataStore(file)
    fmax = data.df.index.max()
    duty = []
    x = []
    y = []
    u = []
    v = []
    for f in range(0, int(fmax), fmax//10):
            points = data.df.loc[f]
            duty.append(int(points.Duty.mean()))
            x.append(points.x.values)
            y.append(points.y.values)
            if type(param) is str:
                u.append(points[param].values.real)
                v.append(points[param].values.imag)
            elif type(param) is tuple:
                u.append(points[param[0]].values)
                v.append(points[param[1]].values)
    return duty, x, y, u, v

def generate_plot_data(files, param):
    duty = []
    x = []
    y = []
    u = []
    v = []
    for file in tqdm(files):
        duty_i, x_i, y_i, u_i, v_i = get_data_attributes(file, param)
        duty += duty_i
        x += x_i
        y += y_i
        u += u_i
        v += v_i
    return pd.DataFrame({'duty': duty, 'x': x, 'y': y, 'u': u, 'v': v})

In [107]:
dimpled_torder = generate_plot_data(dimpled_files, 'torder')

100%|██████████| 100/100 [03:24<00:00,  2.04s/it]


In [110]:
# dimpled_torder = pd.DataFrame({'duty': dimpled_duty, 'x': dimpled_x, 'y': dimpled_y, 'u': dimpled_u, 'v': dimpled_v})
dimpled_torder.to_hdf(f"{save_direc}/dimpled_torder.hdf5", 'df')


/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['x', 'y', 'u', 'v'], dtype='object')]

  encoding=encoding,


In [28]:
dimpled_torder = pd.read_hdf(f"{save_direc}/dimpled_torder.hdf5", 'df')

In [116]:
flat_torder = generate_plot_data(flat_files, 'torder')

100%|██████████| 100/100 [03:15<00:00,  1.96s/it]


In [118]:
# flat_torder = pd.DataFrame({'duty': flat_duty, 'x': flat_x, 'y': flat_y, 'u': flat_u, 'v': flat_v})
flat_torder.to_hdf(f"{save_direc}/flat_torder.hdf5", 'df')

/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['x', 'y', 'u', 'v'], dtype='object')]

  encoding=encoding,


In [12]:
def update_plot(data):
    data = data[1]
    for artist in ax.collections:
        artist.remove()
    ax.quiver(data.x, data.y, data.u, data.v, color=cmap(norm(np.angle(data.u+1j*data.v))), pivot='mid')
    ax.set_title(data.duty)

In [13]:
def create_animation_window():
    fig, (ax, cax) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [50,1]})
    ax.set_aspect('equal')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    cmap = matplotlib.cm.hsv
    norm = matplotlib.colors.Normalize(vmin=-np.pi, vmax=np.pi)
    cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm, orientation='vertical')
    return fig, ax, cmap, norm

In [116]:
data = dimpled_torder
fig, ax, cmap, norm = create_animation_window()
ani = FuncAnimation(fig, update_plot, frames=data.iterrows())
# ani.save("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/dimpled_torder_animation.mp4")

In [119]:
data = flat_torder
fig, ax, cmap, norm = create_animation_window()
ani = FuncAnimation(fig, update_plot, frames=data.iterrows())
# ani.save("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/flat_torder_animation.mp4")

In [9]:
dimpled_horder = generate_plot_data(dimpled_files, ('order_r_long', 'order_i_long'))

100%|██████████| 100/100 [03:13<00:00,  1.93s/it]


In [10]:
dimpled_horder.to_hdf(f"{save_direc}/dimpled_horder.hdf5", 'df')

/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['x', 'y', 'u', 'v'], dtype='object')]

  encoding=encoding,


In [8]:
flat_horder = generate_plot_data(flat_files, ('order_r_long', 'order_i_long'))
flat_horder.to_hdf(f"{save_direc}/flat_horder.hdf5", 'df')

100%|██████████| 100/100 [03:14<00:00,  1.94s/it]
/home/ppxjd3/miniconda3/envs/ExperimentsNew/lib/python3.7/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['x', 'y', 'u', 'v'], dtype='object')]

  encoding=encoding,


In [5]:
dimpled_horder = pd.read_hdf(f"{save_direc}/dimpled_horder.hdf5", 'df')
dimpled_torder = pd.read_hdf(f"{save_direc}/dimpled_torder.hdf5", 'df')

flat_horder = pd.read_hdf(f"{save_direc}/flat_horder.hdf5", 'df')
flat_torder = pd.read_hdf(f"{save_direc}/flat_torder.hdf5", 'df')


In [11]:
fig, ax, cmap, norm = create_animation_window()
ani = FuncAnimation(fig, update_plot, frames=dimpled_horder.iterrows())

NameError: name 'create_animation_window' is not defined

In [11]:
ani.save("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/dimpled_horder_animation.mp4")

NameError: name 'ani' is not defined

In [10]:
flat_horder = generate_plot_data(flat_files, ('order_r_long', 'order_i_long'))
flat_horder.to_csv(f"{save_direc}/flat_horder.csv")

100%|██████████| 100/100 [03:14<00:00,  1.95s/it]


In [20]:
%%capture
fig, ax, cmap, norm = create_animation_window()
ani = FuncAnimation(fig, update_plot, frames=flat_horder.iterrows())

In [19]:
ani.save("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/flat_horder_animation.mp4")

### Place both rotational and translational order on one plot

In [6]:
g = 9.81
v2a = 42.46
def d2a(d):
    """Converts duty cycle to acceleration ms^-2"""
    a = v2a * 0.003 * d - v2a * 1.5
    return a

def d2G(d):
    """Converts duty cycle to dimensionless acceleration"""
    return d2a(d) / g

In [57]:
flat_duty_sus = np.loadtxt("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/flat_sus_duty.txt")
flat_horder_sus = np.loadtxt("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/flat_sus_horder.txt")
flat_torder_sus = np.loadtxt("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/flat_sus_torder.txt")
flat_duty_sus = np.arange(flat_duty_sus.max(), flat_duty_sus.min()-1, -1)
flat_horder.duty = np.repeat(flat_duty_sus, 11)
flat_torder.duty = np.repeat(flat_duty_sus, 11)
flat_duty_sus = d2G(flat_duty_sus)

In [9]:
import matplotlib.ticker as tck

In [44]:
def create_quad_window(d, h, t):
    fig = plt.figure(constrained_layout=False, dpi=600)
    gs = fig.add_gridspec(ncols=3, nrows=2, width_ratios=(48, 48, 4), height_ratios=(70, 30))
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    cax = fig.add_subplot(gs[0, 2])
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])
    ax1.set_aspect('equal')
    ax2.set_aspect('equal')
    ax3.set_title(f'$\Gamma={d2G(680):.2f}$')
    ax1.xaxis.set_visible(False)
    ax1.yaxis.set_visible(False)
    ax1.set_title('$\psi_T$')
    ax2.set_title('$\psi_6$')
    ax2.xaxis.set_visible(False)
    ax2.yaxis.set_visible(False)
#     ax3.yaxis.set_visible(False)
#     ax4.yaxis.set_visible(False)
    ax3.plot(d, t, 'c-')
    ax3.set_xlabel('$\Gamma$')
    ax3.set_ylabel('$\chi_T$', c='c')
    ax3.set_yticks([])
    ax4.set_yticks([])
    ax4.set_xlabel('$\Gamma$')
    ax4.set_ylabel('$\chi_6$', c='m')
    ax4.plot(d, h, 'm-')
    ax3line = ax3.axvline(d2G(680), c='g', linestyle='--')
    ax4line = ax4.axvline(d2G(680), c='g', linestyle='--')  
    cmap = matplotlib.cm.hsv
    norm = matplotlib.colors.Normalize(vmin=-np.pi, vmax=np.pi)
    cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm, orientation='vertical', ticks=[-np.pi, 0, np.pi])
    cbar.ax.set_yticklabels(['$-\pi$', '0', '$\pi$'])
    fig.subplots_adjust(left=0, bottom=0.1, right=0.95, top=0.95, wspace=0, hspace=0)
    fig.tight_layout()
    return fig, ax1, ax2, ax3, ax3line, ax4line, cmap, norm
    

In [11]:
def update_quad_window(data):
    rot, tra = data
    for artist in ax1.collections:
        artist.remove()
    for artist in ax2.collections:
        artist.remove()
    ax1.quiver(tra.x, tra.y, tra.u, tra.v, color=cmap(norm(np.angle(tra.u+1j*tra.v))), pivot='mid')
    ax2.quiver(rot.x, rot.y, rot.u, rot.v, color=cmap(norm(np.angle(rot.u+1j*rot.v))), pivot='mid')
    
    duty = rot.duty
    ax3.set_title(f'$\Gamma={d2G(duty):.2f}$')
    ax3line.set_xdata(d2G(duty))
    ax4line.set_xdata(d2G(duty))
        

In [12]:
def get_double_data_flat():
    for l in range(len(flat_horder)):
        yield (flat_horder.loc[l], flat_torder.loc[l])

In [13]:
def get_double_data_dimple():
    for l in range(len(dimpled_horder)):
        yield (dimpled_horder.loc[l], dimpled_torder.loc[l])

In [26]:
plt.plot(flat_duty_sus)

In [54]:
flat_duty = np.repeat(flat_duty_sus, 11)

In [55]:
plt.plot(flat_duty)
flat_horder.duty.plot()

<AxesSubplot:>

In [59]:
fig, ax1, ax2, ax3, ax3line, ax4line, cmap, norm = create_quad_window(flat_duty_sus, flat_horder_sus, flat_torder_sus)
ani = FuncAnimation(fig, update_quad_window, frames=get_double_data_flat, save_count=len(flat_horder))
ani.save(f'{save_direc}/flat_animation.mp4', fps=30)

In [60]:
d_duty_sus = np.loadtxt("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/dimpled_sus_duty.txt")
d_horder_sus = np.loadtxt("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/dimpled_sus_horder.txt")
d_torder_sus = np.loadtxt("/media/data/Data/FirstOrder/Susceptibility/Dataset2Figures/dimpled_sus_torder.txt")
d_duty_sus = np.arange(d_duty_sus.max(), d_duty_sus.min()-1, -1)
dimpled_horder.duty = np.repeat(d_duty_sus, 11)
dimpled_torder.duty = np.repeat(d_duty_sus, 11)
d_duty_sus = d2G(d_duty_sus)

In [61]:
fig, ax1, ax2, ax3, ax3line, ax4line, cmap, norm = create_quad_window(d_duty_sus, d_horder_sus, d_torder_sus)

ani = FuncAnimation(fig, update_quad_window, frames=get_double_data_dimple, save_count=len(dimpled_horder))
ani.save(f'{save_direc}/dimpled_animation.mp4', fps=30)